## 리랭킹
문서 후처리는 초기 검색 알고리즘에 의해 검색된 문서들을 대상으로, 보다 정교한 방법을 사용하여 문서의 관련성을 재평가하고 순위를 재조정하는 과정입니다. 이 과정에서 리랭킹(Reranking)은 핵심적인 역할을 합니다. 리랭킹을 통해 초기 검색 알고리즘의 한계를 보완하고 검색 결과의 정확도를 높일 수 있습니다.  


먼저, 희소 검색(BM-25와 같은 키워드 기반의 검색) 혹은 밀집 검색(bge-m3와 같은 임베딩 검색)과 같은 초기 검색 알고리즘을 사용합니다. 이는 마치 도서관에서 책의 제목, 저자 이름, 또는 주제와 관련된 키워드로 빠르게 관련 서가를 찾아가는 것과 같습니다.  

예를 들어, "주식 시장 투자 전략"이라는 주제로 검색한다면, 희소 검색은 "주식", "투자", "전략" 등의 키워드가 포함된 책들을 신속하게 찾아냅니다. 동시에 밀집 검색은 "증권 분석 방법", "금융 시장 예측 기법" 등 직접적인 키워드 매칭은 없지만 의미적으로 관련된 책들도 발견합니다. 이 과정은 빠르지만, 관련성이 떨어지는 책들(예: "주식 시장 역사" 혹은 "금융 시장 예측 기법")도 함께 선별될 수 있습니다. 결과적으로, 초기 검색 단계에서는 다음과 같은 책들이 선별되어 다음 단계인 리랭킹 과정으로 전달됩니다:

1.	희소 검색 결과: "현대 주식 투자 전략", "주식 시장 역사"
2.	밀집 검색 결과: "증권 분석 방법", "금융 시장 예측 기법"

이제 선별된 책들에 대해 트랜스포머 기반의 리랭커 모델을 적용해봅시다. 트랜스포머 기반 리랭커는 단순한 키워드 매칭이나 의미적 유사성을 넘어서, 책의 전체적인 맥락과 잠재적 가치를 평가할 수 있습니다. 따라서 리랭커의 작동 과정은 금융 전문가가 각 책의 내용을 직접 검토하고 주제와의 관련성을 더 깊이 평가하는 과정과 유사하다 할 수 있습니다.  

리랭커는 각 책의 목차, 서문, 주요 내용을 자세히 검토하여 "주식 시장 투자 전략"이라는 주제와의 관련성을 평가합니다. 이 과정에서 각 책에 대해 logit 값을 산출하며, 이 값이 높을수록 주제와의 관련성이 높다고 판단합니다. 예를 들어, 다음과 같은 logit 값이 산출될 수 있습니다:  

1.	"현대 주식 투자 전략": 3.5
2.	"주식 시장 역사": 1.2
3.	"증권 분석 방법": 2.8
4.	"금융 시장 예측 기법": 2.1  

이제 리랭커는 미리 설정된 threshold 값(예: 2.5)을 적용하여 최종 선별을 수행합니다. 이 threshold를 넘는 책들만 최종 결과로 선정됩니다. 따라서 최종 결과는 다음과 같이 두 권의 책만 남게 됩니다:
1.	"현대 주식 투자 전략" (logit: 3.5)
2.	"증권 분석 방법" (logit: 2.8)  

이러한 초기 검색과 리랭킹의 두 단계 접근 방식을 통해, 방대한 양의 책 중에서 빠르게 관련 있는 책들을 찾아내고, 그중에서도 가장 적절한 책들을 정확하게 선별할 수 있습니다. 이는 도서관 전체를 처음부터 꼼꼼히 살펴보는 것보다 훨씬 효율적이며, 단순히 키워드 매칭에만 의존하는 것보다 훨씬 정확한 결과를 제공합니다.  

이제, 리랭커의 구체적인 구현 방식을 살펴보겠습니다.

## 1. LLM 기반

고성능 LLM 기반 리랭킹은 Claude나 GPT와 같은 고성능 언어 모델을 활용하여 초기 검색 결과를 재평가하고 순위를 조정하는 방식입니다. 구현 방식은 다음과 같이 매우 간단합니다:  

1.	프롬프트 설계: 질의와 문서를 입력으로 받아 관련성을 평가하는 프롬프트를 작성합니다.
2.	LLM 호출: 각 문서에 대해 LLM을 호출하여 관련성 점수를 얻습니다.
3.	순위 재조정: 얻은 점수를 기반으로 문서들의 순위를 재조정합니다.
 이 방식은 고성능의 LLM을 활용하기때문에 매우 정확한 관련성 평가가 가능합니다. 실제로 ChatGPT-4를 리랭킹에 활용한 결과, TREC, BEIR과 같은 다양한 평가 데이터셋에서 BM25, monoBERT, monoT5, Cohere Rerank 등의 기존 모델들을 크게 앞서는 성능을 내었다는 연구 결과도 있습니다 . 또한, 광범위하게 학습된 LLM의 특성 상, 추가적인 학습 없이 다양한 도메인의 질의 유형에 쉽게 적용할 수 있다는 점도 이 방식의 장점으로 꼽을 수 있습니다. 그리고 필요 시 관련성 평가 점수의 추론 근거를 제공할 수 있기 때문에, 설명 가능한 AI(Explainable AI) 측면에서 결과 해석이 용이하다는 특징이 있습니다.
 반면, 많은 리소스를 필요로 하기때문에 계산비용이 높다는 점과, 느린 처리속도는 단점으로 꼽힙니다.  

랭체인을 활용한 구현 코드는 다음과 같습니다. 앞선 챕터에서 다룬 밀집 검색 방식의 FAISS 리트리버에 리랭킹을 적용하는 코드를 구현하도록 하겠습니다.
 먼저, 필요한 라이브러리들을 설치합니다. FAISS 라이브러리의 경우, GPU 사용이 가능한 환경이라면 faiss-gpu를, 그렇지않다면 faiss-cpu를 설치하도록 합니다.


In [ ]:
!pip install langchain langchain_openai langchain_community pypdf faiss-gpu

In [ ]:
import os

# OpenAI에서 발급받은 API Key를 'sk-...' 부분에 기입합니다.
os.environ["OPENAI_API_KEY"] = "여러분의 Key값"

사용할 문서를 불러온 뒤 적절한 크기로 분할합니다. 앞선 검색 알고리즘 예시들과 동일한 코드를 사용할 것이므로, 해당 부분을 숙지하였다면 아래 설명은 간단히 넘어가도록 합니다.  

우선 PyPDFLoader와 RecursiveCharacterTextSplitter를 임포트합니다. PyPDFLoader는 PDF 파일을 불러오는 기능을, RecursiveCharacterTextSplitter는 긴 텍스트를 작은 단위로 분할하는 역할을 수행합니다.  

file_path 변수에 PDF 파일의 위치를 지정한 다음, 이를 사용해 PyPDFLoader 객체인 loader를 만듭니다. 이 loader는 지정된 PDF 파일을 불러올 준비를 합니다.  

그 다음, RecursiveCharacterTextSplitter 객체인 doc_splitter를 생성합니다. 이때 chunk_size 매개변수를 2000으로 설정해 각 텍스트 조각의 최대 길이를 2000자로 제한하고, chunk_overlap을 200으로 지정해 각 조각 사이에 200자의 겹침을 허용합니다. 이러한 겹침은 문맥의 연속성을 보장하는 데 도움이 됩니다.
마지막으로, loader의 load_and_split 메서드를 실행하여 PDF를 읽고 doc_splitter를 통해 텍스트를 분할합니다. 이 과정의 결과물은 docs 변수에 저장되며, 이는 나눠진 텍스트 조각들의 목록입니다.


In [ ]:
import urllib.request
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings

ModuleNotFoundError: No module named 'langchain_community'

In [ ]:
urllib.request.urlretrieve("https://github.com/chatgpt-kr/openai-api-tutorial/raw/main/ch06/2023_%EB%B6%81%ED%95%9C%EC%9D%B8%EA%B6%8C%EB%B3%B4%EA%B3%A0%EC%84%9C.pdf", filename="2023_북한인권보고서.pdf")

loader = PyPDFLoader('2023_북한인권보고서.pdf')

In [ ]:
doc_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap = 100)

docs = loader.load_and_split(doc_splitter)

이어서 밀집검색을 위한 FAISS DB와 리트리버를 구축합니다. 앞선 밀집 검색의 예시 코드와 동일한 코드를 사용하므로, 해당 부분을 이미 숙지하였다면 아래 설명 역시 간단히 넘어가도록 합니다.  

FAISS DB와 리트리버를 구축은 세 부분으로 진행합니다. 첫 번째는 문서의 벡터화이고, 두 번쨰는 데이터베이스 생성 및 저장, 그리고 마지막으로 리트리버 생성 부분입니다.  

우선 OpenAI의 임베딩 모델을 사용하여 문서들을 밀집 벡터로 변환합니다. 이를 위해 OpenAIEmbeddings 클래스의 인스턴스를 생성하고, model 매개변수로 "text-embedding-3-large"를 지정합니다. 이 embedding 객체는 텍스트를 고차원 벡터로 변환하는 역할을 합니다.


In [ ]:
embedding = OpenAIEmbeddings(model="text-embedding-3-large")

다음으로 FAISS 데이터베이스를 생성하고 저장합니다. FAISS.from_documents 메서드를 사용하여 앞서 분할한 docs와 생성한 embedding 객체를 인자로 전달합니다. 이 과정을 통해 faiss_store라는 FAISS 벡터 저장소가 생성됩니다. 생성된 faiss_store는 save_local 메서드를 통해 "/content/DB" 경로에 저장됩니다.

In [ ]:
# FAISS 라이브러리 임포트
from langchain_community.vectorstores import FAISS

# FAISS 벡터스토어 생성
faiss_store = FAISS.from_documents(docs, embedding)

# FAISS 벡터스토어 저장
persist_directory = "/content/DB"
faiss_store.save_local(persist_directory)

이제, 저장된 FAISS DB를 다시 로드합니다. FAISS.load_local 메서드를 사용하여 저장된 데이터베이스를 불러옵니다. 이때 embeddings 매개변수로 앞서 생성한 embedding 객체를 전달하고, allow_dangerous_deserialization 매개변수를 True로 설정하여 안전하지 않은 역직렬화를 허용합니다. 로드된 벡터 데이터베이스는 vectordb 변수에 할당됩니다.

In [ ]:
vectordb = FAISS.load_local(persist_directory, embeddings=embedding, allow_dangerous_deserialization=True)

이제 LLM기반 리랭킹 알고리즘을 생성합니다. 이 알고리즘은 GPT-4o 모델을 활용하여 초기 검색 결과의 관련성을 평가하고 재정렬합니다.
먼저, 다음 라이브러리와 모듈들을 임포트합니다
-	BaseModel, Field: Pydantic 라이브러리에서 제공하는 클래스로, 데이터 모델을 정의하고 검증하는 데 사용됩니다.
-	PromptTemplate: 랭체인에서 제공하는 클래스로, 프롬프트 템플릿을 생성하는 데 사용됩니다.
-	Document: 랭체인의 문서 클래스로, 텍스트 내용과 메타데이터를 포함하는 문서 객체를 표현합니다.
-	List, Dict, Any, Tuple: 파이썬의 typing 모듈에서 제공하는 타입 힌트로, 함수의 입력과 출력 타입을 명시하는 데 사용됩니다.
-	ChatOpenAI: 랭체인에서 ChatGPT 모델을 사용하기 위한 클래스입니다.
-	dedent: textwrap 모듈의 함수로, 문자열의 들여쓰기를 제거하는 데 사용됩니다.
-	JsonOutputParser: 랭체인에서 제공하는 클래스로, LLM의 출력을 JSON 형식으로 파싱하는 데 사용됩니다.


In [ ]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain import PromptTemplate
from langchain.docstore.document import Document
from typing import List, Dict, Any, Tuple
from langchain.chat_models import ChatOpenAI
from textwrap import dedent
from langchain_core.output_parsers import JsonOutputParser

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


 이제 LLM 기반 리랭킹 알고리즘을 구현하겠습니다.

우선, RelevanceScore 클래스를 정의합니다. 이 클래스는 Pydantic의 BaseModel을 상속받아 관련성 점수를 나타내는 데이터 모델을 정의합니다. 이 클래스의 역할은 리랭커 LLM의 출력이 실수 형태로 나오도록 강제하는 것입니다. 이를 통해 LLM이 질문과 문서의 연관성 점수를 실수로 표현할 수 있도록 합니다.  

다음으로 reranking_documents 함수를 생성합니다. 이 함수는 사용자의 질문과 초기 검색 문서들을 입력받고, 리랭킹이 수행된 문서 리스트를 반환하는 역할을 수행합니다.

먼저 JsonOutputParser를 사용하여 LLM의 출력을 RelevanceScore 객체로 파싱할 수 있도록 준비합니다. 그 다음, PromptTemplate을 이용해 LLM에게 전달할 프롬프트를 정의합니다. 이 프롬프트는 LLM에게 주어진 문서가 질문과 얼마나 관련이 있는지 1점부터 10점까지의 점수로 평가하도록 요청하도록 설계되었습니다.
LLM 모델로는 gpt-4o를 사용하며, 온도를 0으로 설정하여 일관된 출력을 얻습니다.
이제 프롬프트, LLM, 파서를 연결하여 체인을 구성합니다. 이 체인은 입력된 질문과 문서에 대해 관련성 점수를 계산합니다.  

이렇게 구성한 체인을 입력받은 각 문서에 실행하여 점수를 얻습니다. 오류가 발생할 경우 기본 점수 5점을 사용합니다. 모든 문서에 대해 점수를 매긴 후, 점수에 따라 내림차순으로 정렬합니다. 마지막으로 상위 n개의 문서만 선택하여 반환합니다.


In [ ]:
class RelevanceScore(BaseModel):
    relevance_score: float = Field(description="문서가 쿼리와 얼마나 관련이 있는지를 나타내는 점수.")

def reranking_documents(query: str, docs: List[Document], top_n: int = 2) -> List[Document]:
    parser = JsonOutputParser(pydantic_object=RelevanceScore)
    human_message_prompt = PromptTemplate(
        template = """
        1점부터 10점까지 점수를 매겨, 다음 문서가 질문이 얼마나 관련이 있는지 평가해주세요. 단순히 키워드가 일치하는 것이 아니라 쿼리의 구체적인 맥락과 의도를 고려하세요.
        {format_instructions}
        question: {query}
        document: {doc}
        relevance_score:""",
        input_variables=["query", "doc"],
        partial_variables={"format_instructions": parser.get_format_instructions()}
    )

    llm = ChatOpenAI(temperature=0, model_name="gpt-4o", max_tokens=3000)
    chain = human_message_prompt | llm | parser
    scored_docs = []
    for doc in docs:
        input_data = {"query": query, "doc": doc.page_content}
        try:
            score = chain.invoke(input_data)['relevance_score']
            score = float(score)
        except Exception as e:
            print(f"오류 발생: {str(e)}")
            default_score = 5  # 기본 점수를 5점으로 설정
            print(f"기본 점수 {default_score}점을 사용합니다.")
            score = default_score
        scored_docs.append((doc, score))

    reranked_docs = sorted(scored_docs, key=lambda x: x[1], reverse=True)
    return [doc for doc, _ in reranked_docs[:top_n]]

 이제 리랭킹 알고리즘을 적용한 문서 검색을 수행해보겠습니다. 문서의 매우 지엽적인 내용인 “2022년 영업손실”에 대한 관련문서를 찾은 뒤, 리랭킹을 통해 관련 문서만을 남기는 작업을 수행합니다.
먼저, 사용자의 질문(query)를 정의합니다. 이후 similarity_search메서드를 사용해서 앞서 정의한 FAISS 벡터 DB를 기반으로 밀집검색을 수행합니다. 이 때 검색할 문서의 수는 4개로 지정합니다(k=4).
이후 reranking_documents함수에 사용자의 질문(query)와 초기 검색 결과(initial_docs)를 입력하여 리랭킹된 문서 리스트(reranked_docs)를 반환하도록 합니다.


In [ ]:
query = "19년 말 평양시 소재 기업소에서 달마다 배급받은 음식"
initial_docs = vectordb.similarity_search(query, k=4)
reranked_docs = reranking_documents(query, initial_docs)

<ipython-input-11-1d6579297884>:17: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0, model_name="gpt-4o", max_tokens=3000)


이제, 사용자의 질문과 초기 검색결과, 그리고 리랭킹된 문서 리스트를 각각 출력해보도록 하겠습니다.

In [ ]:
# print first 4 initial documents
print(f"Query: {query}\n\n")

print("Top initial documents:")
for i, doc in enumerate(initial_docs):
    print(f"\nDocument {i+1}:")
    print(doc.page_content)

# Print results
print("\n\nTop reranked documents:")
for i, doc in enumerate(reranked_docs):
    print(f"\nDocument {i+1}:")
    print(doc.page_content)  # Print each documents

Query: 19년 말 평양시 소재 기업소에서 달마다 배급받은 음식


Top initial documents:

Document 1:
화 또는 쌀이나 기름 등 현물로 지급하였다고 한다. 2019년 평양
의 외화벌이 사업소에서는 보수 50달러를 월 2회로 나누어 현금으
로 지급하였다고 하는 사례가 있었고, 평양 외화벌이 식당에서는 매

Document 2:
파악되었다. 따라서 기관·기업소의 상황에 따라 식량배급량, 주기, 
곡식종류에 상당한 차이가 있는 것으로 나타났다. 외화벌이 기관 등
에는 식량배급이 원활하게 이뤄지고 있었다는 증언이 수집되었다. 
2019년 평양시에서 기업소 운전원으로 일하였던 노동자는 매월 쌀·
설탕·기름·야채·돼지고기 등을 배급받아 식량이 부족하지 않았다는 
증언과 2019년 중앙당 산하의 기업소에서 매월 쌀 6㎏ 정도, 기름 5
ℓ, 설탕 2㎏, 맛내기 2봉지, 돼지고기 2㎏, 닭고기 1마리 정도 받았

Document 3:
가배급을 선택하고, 잘사는 기업소들은 기업소 자체 배급을 선택합
니 다. 세대주가 직장에 다닐 경우 세대주만 직장에서 배급을 받고 
가족들은 국가배급소에서 배급을 받습니다. 평양시와 자강도는 대
체로 다 줬는데 다른 지역은 배급이 잘 안되고 배급제가 없어졌다는 
소리를 들었습니다. ”
국가배급의 주기, 양, 곡물의 종류 등에서 평양시와 지방의 차이
가 크게 나고 있었다. 식량배급이 비교적 원활하게 작동하는 지역은 
평양시로 보이는데, 2017년 어머니가 지역배급 대상자로 배급표가

Document 4:
한 달을 생활하기에 부족한 금액이었다고 하였다. 2018년 양강도의 
무역사업소에서는 1년치 노동 보수와 배급을 한 번에 지급하였다고 
하는데, 지급된 금액은 노동자 1명에게 1,800위안으로 약 300만원 
정도였다고 하였다. 2019년 양강도의 합영회사는 노동자에게 매달 
9~12만원의 보수를 지급하고, 1년에 한번 쌀 25kg을 지급하였다는 
진술이 있었다. 또한 2020년 합영회사에서는 보수를 성과만큼 받았

위 문서검색 결과, 초기 4개의 문서 중, 평양 배급에 대해 정확히 언급한 2개의 문서만이 리랭킹을 거친 후 남게 되었음을 알 수 있습니다.

이제 최종적으로 밀집 검색과 리랭킹을 거친 문서를 사용하여 답변까지 하는 시스템을 구현해보겠습니다.

먼저, 아래 모듈을 임포트합니다.
-	BaseRetriever: 랭체인에서 제공하는 기본 검색기 클래스입니다. 사용자 정의 검색기를 구현할 때 상속받아서 랭체인의 기본적인 검색기 기능을 사용할 수 있도록 합니다.
-	RetrievalQA: 랭체인에서 제공하는 클래스로, 검색 기반 질문-답변 시스템을 구현하는 데 사용됩니다. 이 클래스는 주어진 질문에 대해 관련 문서를 검색하고, 그 문서를 바탕으로 답변을 생성하는 기능을 제공합니다.


In [ ]:
from langchain_core.retrievers import BaseRetriever
from langchain.chains import RetrievalQA

이제 커스텀 리트리버 체인을 생성하여 리랭킹 알고리즘을 검색 과정에 통합해보겠습니다.
 먼저 CustomRetriever 클래스를 정의합니다. 이 커스텀 리트리버는 기본 검색기능을 재정의하여 리랭킹 과정을 포함하도록 설계되었습니다. 이 클래스는 아래 두 클래스를 상속받습니다.
-	BaseRetriever: 랭체인에서 제공하는 기본 검색기 클래스로, 검색기능을 수행하는 데 필요한 기본 메서드들을 제공합니다.
-	BaseModel: Pydantic의 기본 모델 클래스로, 데이터 검증과 설정을 용이하게 합니다.  

다음으로, 검색에 사용할 벡터스토어를 클래스 변수로 정의합니다. 각 항목은 다음과 같습니다.
-	vectorstore: 벡터 임베딩된 문서들을 저장하고 있는 벡터스토어 객체입니다. 여기서는 앞서 생성한 FAISS 벡터스토어를 사용할 예정입니다.
-	Any: 벡터스토어의 타입을 Any로 지정하여 어떤 형태의 벡터스토어도 받을 수 있도록 합니다.
-	Field: Pydantic에서 제공하는 필드 설정으로, 변수에 대한 추가 메타데이터나 검증 옵션을 지정할 수 있습니다.

그리고 내부 클래스인 Config를 정의하여 Pydantic 모델 설정에서 임의의 타입을 허용하도록 구성합니다.
-	arbitrary_types_allowed = True: Pydantic은 기본적으로 표준 Python 타입만 허용하지만, 이 설정을 True로 변경하면 사용자 정의 클래스나 외부 라이브러리의 객체와 같은 임의의 타입도 허용하게 됩니다. 이는 vectorstore 속성이 FAISS와 같은 외부 라이브러리의 객체이기 때문에, Pydantic의 타입 검증 과정에서 오류를 방지하기 위해 필요한 설정입니다.  

이제 get_relevant_documents 메서드를 재정의하여 검색과 리랭킹 과정을 구현합니다. 각 항목은 다음과 같습니다.
-	query: str: 사용자의 질문 문자열입니다.
-	num_docs: int = 2: 리랭킹 후 반환할 문서의 수를 지정하는 매개변수로, 기본값은 2입니다.
-	initial_docs: 벡터스토어에서 쿼리에 대한 초기 검색 결과를 저장합니다. 여기서는 k=4로 설정하여 상위 4개의 유사한 문서를 검색합니다.
-	reranking_documents: 앞서 정의한 리랭킹 함수로, 초기 검색 결과와 질문을 입력받아 관련성이 높은 문서들을 재정렬하고 상위 num_docs개의 문서를 반환합니다.


In [ ]:
from typing import Any, List
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.retrievers import BaseRetriever
from langchain_core.documents import Document

class CustomRetriever(BaseRetriever):
    vectorstore: Any = None

    def __init__(self, **kwargs):
        super().__init__()
        self.vectorstore = kwargs.get("vectorstore")

    def get_relevant_documents(self, query: str, num_docs=2) -> List[Document]:
        initial_docs = self.vectorstore.similarity_search(query, k=4)
        return reranking_documents(query, initial_docs, top_n=num_docs)

    async def aget_relevant_documents(self, query: str) -> List[Document]:
        raise NotImplementedError("Async retrieval not implemented")

<ipython-input-15-cb6d1fa7d253>:6: DeprecationWarning: Retrievers must implement abstract `_get_relevant_documents` method instead of `get_relevant_documents`
  class CustomRetriever(BaseRetriever):
<ipython-input-15-cb6d1fa7d253>:6: DeprecationWarning: Retrievers must implement abstract `_aget_relevant_documents` method instead of `aget_relevant_documents`
  class CustomRetriever(BaseRetriever):


이제 커스텀 리트리버 인스턴스와 LLM을 생성하고, 이를 결합하여 최종적인 RetrievalQA 체인을 구성하겠습니다.
먼저, 앞서 정의한 CustomRetriever 클래스를 기반으로 인스턴스를 생성합니다. 앞서 생성한 FAISS 벡터스토어 vectordb를 vectorstore 매개변수로 전달합니다.
다음으로, 답변 생성을 위한 LLM 인스턴스를 생성합니다. gpt-4o 모델을 사용하고, temperature는 0.2로 설정해서 모델이 더 일관된 응답을 하도록 유도합니다.
마지막으로, RetrievalQA 클래스의 from_chain_type 메서드를 사용하여 QA 체인을 생성합니다. 이 때 각 파라미터는 다음과 같습니다.
-	llm=llm: 앞서 생성한 LLM 인스턴스를 지정합니다.
-	chain_type="stuff": 문서를 처리하는 방식을 지정하는 매개변수로, "stuff"는 검색된 모든 문서를 하나의 문자열로 결합하여 LLM에 전달하는 방법입니다.
-	retriever=custom_retriever: 앞서 생성한 커스텀 리트리버를 사용하여 관련 문서를 검색합니다.
-	return_source_documents=True: 최종 응답과 함께 사용된 소스 문서들을 반환하도록 설정합니다.


In [ ]:
# custom retriever 인스턴스를 생성합니다.
custom_retriever = CustomRetriever(vectorstore=vectordb)

# 답변용 LLM 인스턴스를 생성합니다.
llm = ChatOpenAI(temperature=0.2, model_name="gpt-4o")

# RetrievalQA 체인을 생성합니다.
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=custom_retriever,
    return_source_documents=True
)

최종적으로 QA 체인을 활용하여 실제 질문에 대한 답변을 얻어보겠습니다. 예시 질문으로 “19년 말 평양시 소재 기업소에서 달마다 배급받은 음식” 라는 질문을 하도록 하겠습니다.

In [ ]:
qa_chain.invoke("19년 말 평양시 소재 기업소에서 달마다 배급받은 음식")

{'query': '19년 말 평양시 소재 기업소에서 달마다 배급받은 음식',
 'result': '2019년 말 평양시 소재 기업소에서 일하던 노동자는 매월 쌀, 설탕, 기름, 야채, 돼지고기 등을 배급받았다고 증언했습니다. 구체적으로는 쌀 6㎏ 정도, 기름 5ℓ, 설탕 2㎏, 맛내기 2봉지, 돼지고기 2㎏, 닭고기 1마리를 받았다고 합니다.',
 'source_documents': [Document(metadata={'source': '2023_북한인권보고서.pdf', 'page': 252}, page_content='파악되었다. 따라서 기관·기업소의 상황에 따라 식량배급량, 주기, \n곡식종류에 상당한 차이가 있는 것으로 나타났다. 외화벌이 기관 등\n에는 식량배급이 원활하게 이뤄지고 있었다는 증언이 수집되었다. \n2019년 평양시에서 기업소 운전원으로 일하였던 노동자는 매월 쌀·\n설탕·기름·야채·돼지고기 등을 배급받아 식량이 부족하지 않았다는 \n증언과 2019년 중앙당 산하의 기업소에서 매월 쌀 6㎏ 정도, 기름 5\nℓ, 설탕 2㎏, 맛내기 2봉지, 돼지고기 2㎏, 닭고기 1마리 정도 받았'),
  Document(metadata={'source': '2023_북한인권보고서.pdf', 'page': 249}, page_content='가배급을 선택하고, 잘사는 기업소들은 기업소 자체 배급을 선택합\n니 다. 세대주가 직장에 다닐 경우 세대주만 직장에서 배급을 받고 \n가족들은 국가배급소에서 배급을 받습니다. 평양시와 자강도는 대\n체로 다 줬는데 다른 지역은 배급이 잘 안되고 배급제가 없어졌다는 \n소리를 들었습니다. ”\n국가배급의 주기, 양, 곡물의 종류 등에서 평양시와 지방의 차이\n가 크게 나고 있었다. 식량배급이 비교적 원활하게 작동하는 지역은 \n평양시로 보이는데, 2017년 어머니가 지역배급 대상자로 배급표가')]}

 코드의 작동 결과, "19년 말 평양시 소재 기업소에서 달마다 배급받은 음식"라는 질문에 대해 정확히 "쌀 6㎏ 정도, 기름 5ℓ, 설탕 2㎏, 맛내기 2봉지, 돼지고기 2㎏, 닭고기 1마리 정도"이라고 답변했습니다. 이는 리랭킹 과정을 거쳐 선별된 두 개의 관련 문서에서 정확한 정보를 추출한 결과로, LLM 기반 리랭킹이 질문의 의도를 정확히 파악하고 관련성 높은 문서를 효과적으로 선별하였음을 보여줍니다.

## 크로스 인코더 기반 리랭킹

크로스인코더 모델 기반 리랭킹은 언어 모델 중, BERT와 같은 인코더 기반 모델을 사용하는 방식입니다. 크로스인코더 모델은 일반적으로 두 텍스트를 결합한 후 전체 시퀀스에 대한 단일 점수를 출력하며, 이 점수가 두 문서의 관련성을 나타냅니다. 이 과정에서 BERT와 같은 모델의 [CLS] 토큰을 활용합니다. [CLS] 토큰은 입력 시퀀스의 시작을 나타내는 특수 토큰으로, 모델 학습 과정에서 전체 입력의 의미를 포괄하는 표현을 학습하게 됩니다. 즉, 입력 시퀀스로 들어간 "질문"과 "문서"의 포괄적인 의미와 관계를 대표하게 되며, 이를 신경망 처리를 통해 변환시켜 최종적인 관련성 점수로 사용합니다. 이 방식으로 크로스인코더는 질문과 문서 사이의 복잡한 의미적 관계를 고려하여 매우 정확한 관련성 평가를 수행할 수 있습니다. 구체적인 과정은 다음과 같습니다:

1. 입력 구성: 쿼리와 문서를 "[CLS] 쿼리 [SEP] 문서 [SEP]" 형태로 결합합니다. [SEP] 토큰은 두 텍스트를 구분하는 역할을 합니다.  
2. 인코딩: 결합된 입력을 모델에 통과시켜 각 토큰에 대한 임베딩을 얻습니다.  
3. [CLS] 토큰 활용: 최종 층의 [CLS] 토큰 임베딩을 추출합니다. 이 임베딩은 쿼리와 문서 간의 관계를 포괄적으로 표현합니다.  
4. 점수 계산: [CLS] 토큰 임베딩에 선형 층을 적용하여 최종 관련성 점수를 산출합니다.  


```
Bi-Encoder VS Cross-Encoder
리랭킹에 대한 주제로 자주 등장하는 두 인코더를 살펴보겠습니다.


Bi-encoder는 앞선 챕터에서 다룬 밀집 검색(Dense retrieval)에서 사용되는 인코딩 방식으로, 밀집 검색의 기반이 되는 벡터 표현을 생성합니다. 질문 A와 문서 B가 있을 때, 이들을 임베딩을 통해 동일한 크기의 벡터로 만든 뒤, Cosine similarity와 같은 유사도 분석을 통해 벡터간의 거리를 수치화하는 방식입니다. 두 벡터간의 거리가 가깝다면 질문과 문서가 의미적으로 유사하다고 판단하고, 멀다면 관련성이 낮다고 판단합니다.
이에 반해 Cross-encoder는 질문 A와 문서 B가 있을 때, 이들을 트랜스포머 기반의 인코더 모델에 함께 투입하여, 둘 사이의 관련성을 나타내는 분류 점수(Classification score)를 직접 얻습니다.
이 두 방식의 특징을 속도와 정확도 측면에서 비교해보겠습니다.

1. 속도
Bi-encoder는 Cross-encoder에 비해 일반적으로 더 빠른 속도를 보입니다. 이는 두 방식의 작동 원리 차이에서 기인합니다.
- Bi-encoder: 각 문장을 독립적으로 인코딩합니다. 예를 들어, 100,000개의 문장이 있다면 100,000번의 인코딩 작업만 수행하면 됩니다.
- Cross-encoder: 모든 가능한 문장 쌍을 동시에 인코딩합니다. 100,000개의 문장이 있을 경우, 조합 공식에 의해 약 50억 개(정확히 4,999,950,000개)의 쌍을 인코딩해야 합니다.
이러한 작동 방식의 차이로 인해, 대규모 데이터셋에서 Cross-encoder는 Bi-encoder에 비해 현저히 느린 속도를 보입니다. 특히 수천 개 이상의 문장을 비교해야 하는 경우 Cross-encoder의 속도 저하는 더욱 두드러집니다.

2. 정확도
 Cross-encoder가 Bi-encoder에 비해 일반적으로 더 높은 정확도를 보입니다.
- Bi-encoder: 각 문장을 독립적으로 인코딩하기 때문에, 두 문장 간의 관계를 직접적으로 고려하지 않습니다. 이는 빠른 처리를 가능하게 하지만, 문장 간의 복잡한 상호작용을 포착하는 데 한계가 있을 수 있습니다.
- Cross-encoder: 두 문장을 동시에 인코딩하여 하나의 임베딩을 생성합니다. 이 과정에서 두 문장 간의 관계를 직접적으로 모델링할 수 있어, 더 정확한 분류나 유사도 평가가 가능합니다.

 결론적으로, 속도 측면에서는 Bi-encoder가 유리하고 정확도 측면에서는 Cross-encoder가 유리합니다. 따라서 작업의 성격과 요구사항에 따라 적절한 인코더를 선택해야 할 필요가 있습니다.
실제 응용에서는 두 방식을 조합하여 사용하기도 합니다. 예를 들어, Bi-encoder를 사용해 대규모 데이터셋에서 후보군을 빠르게 추려낸 후, Cross-encoder를 사용해 이 후보군 내에서 더 정확한 랭킹을 수행하는 방식입니다. 이를 통해 속도와 정확도 사이의 균형을 맞출 수 있습니다.
```

이 방식에 쓰이는 모델은 앞선 챕터에서 다른 고성능 LLM기반 리랭커보다 낮은 사양의 모델을 사용하기 때문에 필요로 하는 계산 리소스 역시 낮습니다. 또한, 비교적 낮은 비용으로 특정 도메인의 데이터로 파인튜닝하여 리랭킹 성능을 더욱 향상시킬 수 있다는 장점이 있습니다.  

하지만 디코더 기반의 상용 LLM에 비해 사전 학습된 지식의 범위가 제한적이라 추가적인 학습이 필요한 경우가 많으며, 관련성 점수의 추론과정을 자연어로 제공하지 못한다는 점은 단점이 될 수 있습니다.  

구현 코드는 다음과 같습니다. 앞선 고성능 llm기반 리랭커 챕터에서 리랭커 부분만 변환한 코드 예시이므로, 해당 부분을 숙지하였다면 앞 부분의 설명은 가볍게 넘어가도록 합니다.

In [ ]:
urllib.request.urlretrieve("https://github.com/chatgpt-kr/openai-api-tutorial/raw/main/ch06/2023_%EB%B6%81%ED%95%9C%EC%9D%B8%EA%B6%8C%EB%B3%B4%EA%B3%A0%EC%84%9C.pdf", filename="2023_북한인권보고서.pdf")

loader = PyPDFLoader('2023_북한인권보고서.pdf')

In [ ]:
doc_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap = 100)

docs = loader.load_and_split(doc_splitter)

이어서 밀집검색을 위한 FAISS DB와 리트리버를 구축합니다. 앞선 밀집 검색의 예시 코드와 동일한 코드를 사용합니다.
FAISS DB와 리트리버를 구축은 세 부분으로 진행합니다. 첫 번째는 문서의 벡터화이고, 두 번쨰는 데이터베이스 생성 및 저장, 그리고 마지막으로 리트리버 생성 부분입니다.  

우선 OpenAI의 임베딩 모델을 사용하여 문서들을 밀집 벡터로 변환합니다. 이를 위해 OpenAIEmbeddings 클래스의 인스턴스를 생성하고, model 매개변수로 "text-embedding-3-large"를 지정합니다. 이 embedding 객체는 텍스트를 고차원 벡터로 변환하는 역할을 합니다.

In [ ]:
embedding = OpenAIEmbeddings(model="text-embedding-3-large")

다음으로 FAISS 데이터베이스를 생성하고 저장합니다. FAISS.from_documents 메서드를 사용하여 앞서 분할한 docs와 생성한 embedding 객체를 인자로 전달합니다. 이 과정을 통해 faiss_store라는 FAISS 벡터 저장소가 생성됩니다. 생성된 faiss_store는 save_local 메서드를 통해 "/content/DB" 경로에 저장됩니다.

In [ ]:
# FAISS 라이브러리 임포트
from langchain_community.vectorstores import FAISS

# FAISS 벡터스토어 생성
faiss_store = FAISS.from_documents(docs, embedding)
# FAISS 벡터스토어 저장
persist_directory = "/content/DB"
faiss_store.save_local(persist_directory)

저장된 FAISS DB를 다시 로드합니다. FAISS.load_local 메서드를 사용하여 저장된 데이터베이스를 불러옵니다. 이때 embeddings 매개변수로 앞서 생성한 embedding 객체를 전달하고, allow_dangerous_deserialization 매개변수를 True로 설정하여 안전하지 않은 역직렬화를 허용합니다. 로드된 벡터 데이터베이스는 vectordb 변수에 할당됩니다.

In [ ]:
# 저장한 FAISS DB 불러오기
vectordb = FAISS.load_local(persist_directory, embeddings=embedding, allow_dangerous_deserialization=True)

이제 크로스 인코더 기반 리랭킹 알고리즘을 생성합니다. 이 예시에서는 ms-marco-MiniLM-L-12-v2 모델을 사용합니다. 해당 모델은 Microsoft가 개발한 MS MARCO(Microsoft MAchine Reading COmprehension) 데이터셋을 기반으로 학습되었습니다.
먼저, 다음 라이브러리와 모듈들을 임포트합니다
-	BaseModel, Field: Pydantic 라이브러리에서 제공하는 클래스로, 데이터 모델을 정의하고 검증하는 데 사용됩니다.
-	Document: 랭체인의 문서 클래스로, 텍스트 내용과 메타데이터를 포함하는 문서 객체를 표현합니다.
-	List, Dict, Any, Tuple: 파이썬의 typing 모듈에서 제공하는 타입 힌트로, 함수의 입력과 출력 타입을 명시하는 데 사용됩니다.
-	ChatOpenAI: 랭체인에서 ChatGPT 모델을 사용하기 위한 클래스입니다.
-	CrossEncoder: Sentence Transformers 라이브러리에서 제공하는 클래스로, 크로스 인코더 모델을 사용하여 텍스트 쌍의 관련성을 평가하는 데 사용됩니다.
-	BaseRetriever: Langchain 라이브러리의 기본 리트리버 클래스입니다. 이 클래스를 상속받아 사용자 정의 리트리버를 구현할 수 있습니다.
-	RetrievalQA: Langchain에서 제공하는 클래스로, 검색(retrieval)과 질문-답변(question answering)을 결합한 체인을 구현합니다. 이 체인은 주어진 질문에 대해 관련 문서를 검색하고, 그 문서를 바탕으로 답변을 생성합니다.

In [ ]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.docstore.document import Document
from typing import List, Dict, Any, Tuple
from langchain.chat_models import ChatOpenAI
from sentence_transformers import CrossEncoder
from langchain_core.retrievers import BaseRetriever
from langchain.chains import RetrievalQA

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


 이제, 리랭커 모델을 다운받기 위해 CrossEncoder 클래스의 인스턴스를 생성합니다. 이 때, 인자로 'cross-encoder/ms-marco-MiniLM-L-12-v2'를 전달합니다. 이 문자열은 우리가 사용하고자 하는 특정 모델의 식별자입니다.

In [ ]:
crossencoder = CrossEncoder('BAAI/bge-reranker-v2-m3')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/795 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

이제 벡터 저장소와 크로스 인코더를 결합한 하이브리드 검색 알고리즘을 구현하겠습니다.  

우선, Retriever_with_cross_encoder 클래스를 정의합니다. 이 클래스는 BaseRetriever와 BaseModel을 상속받아 하이브리드 검색을 위한 사용자 정의 검색기를 구현합니다. 이 클래스의 주요 역할은 벡터 기반 초기 검색과 크로스 인코더를 이용한 재순위화를 결합하여 더 정확한 문서 검색을 수행하는 것입니다.
다음으로 get_relevant_documents 메서드를 구현합니다. 이 메서드는 사용자의 질문을 입력받아 관련성 높은 문서 리스트를 반환하는 역할을 수행합니다. 메서드의 동작 순서는 다음과 같습니다:  

1.	초기 검색: vectorstore를 사용하여 쿼리와 유사한 k개의 문서를 빠르게 검색합니다. 이는 관련 문서의 후보군을 추려내는 역할을 합니다.
2.	문서-질문 쌍 준비: 검색된 각 문서와 질문을 쌍으로 만듭니다. 이는 크로스 인코더의 입력으로 사용됩니다.
3.	관련성 점수 계산: 크로스 인코더 모델을 사용하여 각 질문-문서 쌍의 관련성 점수를 계산합니다. 이 과정은 초기 검색 결과를 더 정확하게 평가합니다.
4.	정렬: 계산된 점수를 기준으로 문서들을 내림차순으로 정렬합니다. 이를 통해 가장 관련성 높은 문서가 상위에 오도록 합니다.
5.	최종 결과 반환: 정렬된 문서 중 상위 rerank_top_k개의 문서만 선택하여 최종 결과로 반환합니다.


In [ ]:
from typing import Any, List
from langchain_core.retrievers import BaseRetriever
from langchain_core.documents import Document

class Retriever_with_cross_encoder(BaseRetriever):
    vectorstore: Any
    crossencoder: Any
    k: int
    rerank_top_k: int

    @classmethod
    def from_params(
        cls,
        vectorstore: Any,
        crossencoder: Any,
        k: int = 5,
        rerank_top_k: int = 2
    ) -> "Retriever_with_cross_encoder":
        self = cls()
        self.vectorstore = vectorstore
        self.crossencoder = crossencoder
        self.k = k
        self.rerank_top_k = rerank_top_k
        return self

    def get_relevant_documents(self, query: str) -> List[Document]:
        initial_docs = self.vectorstore.similarity_search(query, k=self.k)
        pairs = [[query, doc.page_content] for doc in initial_docs]
        scores = self.crossencoder.predict(pairs)
        scored_docs = sorted(zip(initial_docs, scores), key=lambda x: x[1], reverse=True)
        return [doc for doc, _ in scored_docs[:self.rerank_top_k]]

    async def aget_relevant_documents(self, query: str) -> List[Document]:
        raise NotImplementedError

<ipython-input-25-470b45776512>:5: DeprecationWarning: Retrievers must implement abstract `_get_relevant_documents` method instead of `get_relevant_documents`
  class Retriever_with_cross_encoder(BaseRetriever):
<ipython-input-25-470b45776512>:5: DeprecationWarning: Retrievers must implement abstract `_aget_relevant_documents` method instead of `aget_relevant_documents`
  class Retriever_with_cross_encoder(BaseRetriever):


이제 앞선 예시와 마찬가지로, 리랭킹 알고리즘을 적용한 문서 검색을 수행해보겠습니다.
먼저, Retriever_with_cross_encoder 클래스의 인스턴스를 생성합니다. 이 인스턴스는 벡터 저장소와 크로스 인코더를 결합한 하이브리드 검색을 수행하는 데 사용됩니다. 인스턴스의 각 파라미터 설정은 다음과 같습니다:
1.	vectorstore 설정: 앞서 생성한 vectordb를 벡터 저장소로 지정합니다. 이는 초기 밀집 검색에 사용됩니다.
2.	crossencoder 설정: 미리 준비된 crossencoder 모델을 지정합니다. 이는 검색된 문서의 재순위화에 사용됩니다.
3.	k 값 설정: 초기 밀집 검색에서 반환할 문서의 수를 4로 설정합니다. 이는 벡터 검색을 통해 먼저 4개의 관련 문서를 추출함을 의미합니다.
4.	rerank_top_k 값 설정: 리랭킹 후 최종적으로 반환할 문서의 수를 2로 설정합니다. 즉, 크로스 인코더를 통해 재평가된 문서 중 가장 관련성 높은 2개만을 최종 결과로 선택합니다.
 다음으로, LLM과 체인 인스턴스를 생성합니다. LLM 인스턴스의 파라미터 설정은 다음과 같습니다.
1.	ChatOpenAI 모델 설정: gpt-4o 모델을 사용하여 LLM 인스턴스를 생성합니다.
2.	temperature 설정: 0.2로 설정하여 비교적 일관된 출력을 생성하도록 합니다. 이는 창의성보다는 일관성에 중점을 둔 설정입니다.


In [ ]:
cross_encoder_retriever = Retriever_with_cross_encoder(
    vectorstore=vectordb,
    crossencoder=crossencoder,
    k=4,
    rerank_top_k=2
)

llm = ChatOpenAI(temperature=0.2, model_name="gpt-4o")

RetrievalQA 체인 인스턴스의 파라미터 설정은 다음과 같습니다.
-	llm 설정: 앞서 생성한 ChatOpenAI 모델 인스턴스를 사용합니다.
-	chain_type 설정: "stuff" 방식을 사용합니다. 이는 검색된 모든 문서를 하나의 컨텍스트로 결합하여 LLM에 제공하는 방식입니다.
-	retriever 설정: 앞서 생성한 cross_encoder_retriever를 사용합니다. 이를 통해 하이브리드 검색 결과를 QA 시스템에 활용합니다.
-	return_source_documents 설정: True로 설정하여 답변과 함께 원본 문서도 반환하도록 합니다.


In [ ]:
query = "19년 말 평양시 소재 기업소에서 달마다 배급받은 음식"
initial_docs = vectordb.similarity_search(query, k=4)

for docs in initial_docs:
  print(docs)
  print('---')

page_content='화 또는 쌀이나 기름 등 현물로 지급하였다고 한다. 2019년 평양
의 외화벌이 사업소에서는 보수 50달러를 월 2회로 나누어 현금으
로 지급하였다고 하는 사례가 있었고, 평양 외화벌이 식당에서는 매' metadata={'source': '2023_북한인권보고서.pdf', 'page': 293}
---
page_content='파악되었다. 따라서 기관·기업소의 상황에 따라 식량배급량, 주기, 
곡식종류에 상당한 차이가 있는 것으로 나타났다. 외화벌이 기관 등
에는 식량배급이 원활하게 이뤄지고 있었다는 증언이 수집되었다. 
2019년 평양시에서 기업소 운전원으로 일하였던 노동자는 매월 쌀·
설탕·기름·야채·돼지고기 등을 배급받아 식량이 부족하지 않았다는 
증언과 2019년 중앙당 산하의 기업소에서 매월 쌀 6㎏ 정도, 기름 5
ℓ, 설탕 2㎏, 맛내기 2봉지, 돼지고기 2㎏, 닭고기 1마리 정도 받았' metadata={'source': '2023_북한인권보고서.pdf', 'page': 252}
---
page_content='가배급을 선택하고, 잘사는 기업소들은 기업소 자체 배급을 선택합
니 다. 세대주가 직장에 다닐 경우 세대주만 직장에서 배급을 받고 
가족들은 국가배급소에서 배급을 받습니다. 평양시와 자강도는 대
체로 다 줬는데 다른 지역은 배급이 잘 안되고 배급제가 없어졌다는 
소리를 들었습니다. ”
국가배급의 주기, 양, 곡물의 종류 등에서 평양시와 지방의 차이
가 크게 나고 있었다. 식량배급이 비교적 원활하게 작동하는 지역은 
평양시로 보이는데, 2017년 어머니가 지역배급 대상자로 배급표가' metadata={'source': '2023_북한인권보고서.pdf', 'page': 249}
---
page_content='한 달을 생활하기에 부족한 금액이었다고 하였다. 2018년 양강도의 
무역사업소에서는 1년치 노동 보수와 배급을 한 번에 지급하였다고 
하는데, 지급된 금액은 노동자 1명에게 1,800위안으로 약 300만원 
정

In [ ]:
# RetrievalQA 체인 인스턴스 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=cross_encoder_retriever,
    return_source_documents=True
)

In [ ]:
query = "19년 말 평양시 소재 기업소에서 달마다 배급받은 음식"
result = qa_chain({"query": query})
print(result)

<ipython-input-29-f2ab3a3965f8>:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": query})


{'query': '19년 말 평양시 소재 기업소에서 달마다 배급받은 음식', 'result': '2019년 말 평양시 소재 기업소에서 일하던 노동자는 매월 쌀, 설탕, 기름, 야채, 돼지고기 등을 배급받았다고 합니다. 구체적으로는 쌀 6㎏ 정도, 기름 5ℓ, 설탕 2㎏, 맛내기 2봉지, 돼지고기 2㎏, 닭고기 1마리 정도를 받았다는 증언이 있습니다.', 'source_documents': [Document(metadata={'source': '2023_북한인권보고서.pdf', 'page': 252}, page_content='파악되었다. 따라서 기관·기업소의 상황에 따라 식량배급량, 주기, \n곡식종류에 상당한 차이가 있는 것으로 나타났다. 외화벌이 기관 등\n에는 식량배급이 원활하게 이뤄지고 있었다는 증언이 수집되었다. \n2019년 평양시에서 기업소 운전원으로 일하였던 노동자는 매월 쌀·\n설탕·기름·야채·돼지고기 등을 배급받아 식량이 부족하지 않았다는 \n증언과 2019년 중앙당 산하의 기업소에서 매월 쌀 6㎏ 정도, 기름 5\nℓ, 설탕 2㎏, 맛내기 2봉지, 돼지고기 2㎏, 닭고기 1마리 정도 받았'), Document(metadata={'source': '2023_북한인권보고서.pdf', 'page': 249}, page_content='가배급을 선택하고, 잘사는 기업소들은 기업소 자체 배급을 선택합\n니 다. 세대주가 직장에 다닐 경우 세대주만 직장에서 배급을 받고 \n가족들은 국가배급소에서 배급을 받습니다. 평양시와 자강도는 대\n체로 다 줬는데 다른 지역은 배급이 잘 안되고 배급제가 없어졌다는 \n소리를 들었습니다. ”\n국가배급의 주기, 양, 곡물의 종류 등에서 평양시와 지방의 차이\n가 크게 나고 있었다. 식량배급이 비교적 원활하게 작동하는 지역은 \n평양시로 보이는데, 2017년 어머니가 지역배급 대상자로 배급표가')]}


In [ ]:
print(f"\n질문: {query}")
print(f"답변: {result['result']}")
print("\n답변 근거 문서:")
for i, doc in enumerate(result["source_documents"]):
    print(f"\nDocument {i+1}:")
    print(doc.page_content)  # Print each document


질문: 19년 말 평양시 소재 기업소에서 달마다 배급받은 음식
답변: 2019년 말 평양시 소재 기업소에서 일하던 노동자는 매월 쌀, 설탕, 기름, 야채, 돼지고기 등을 배급받았다고 합니다. 구체적으로는 쌀 6㎏ 정도, 기름 5ℓ, 설탕 2㎏, 맛내기 2봉지, 돼지고기 2㎏, 닭고기 1마리 정도를 받았다는 증언이 있습니다.

답변 근거 문서:

Document 1:
파악되었다. 따라서 기관·기업소의 상황에 따라 식량배급량, 주기, 
곡식종류에 상당한 차이가 있는 것으로 나타났다. 외화벌이 기관 등
에는 식량배급이 원활하게 이뤄지고 있었다는 증언이 수집되었다. 
2019년 평양시에서 기업소 운전원으로 일하였던 노동자는 매월 쌀·
설탕·기름·야채·돼지고기 등을 배급받아 식량이 부족하지 않았다는 
증언과 2019년 중앙당 산하의 기업소에서 매월 쌀 6㎏ 정도, 기름 5
ℓ, 설탕 2㎏, 맛내기 2봉지, 돼지고기 2㎏, 닭고기 1마리 정도 받았

Document 2:
가배급을 선택하고, 잘사는 기업소들은 기업소 자체 배급을 선택합
니 다. 세대주가 직장에 다닐 경우 세대주만 직장에서 배급을 받고 
가족들은 국가배급소에서 배급을 받습니다. 평양시와 자강도는 대
체로 다 줬는데 다른 지역은 배급이 잘 안되고 배급제가 없어졌다는 
소리를 들었습니다. ”
국가배급의 주기, 양, 곡물의 종류 등에서 평양시와 지방의 차이
가 크게 나고 있었다. 식량배급이 비교적 원활하게 작동하는 지역은 
평양시로 보이는데, 2017년 어머니가 지역배급 대상자로 배급표가


크로스인코더는 BERT와 같은 인코더 기반 트랜스포머를 사용하여 고성능 LLM에 비해 빠른 처리 속도를 제공합니다. 그러나 정확도 면에서는 고성능 LLM 기반 방식에 비해 다소 떨어질 수 있습니다.  

따라서 리랭킹 방식의 선택은 작업의 특성과 요구사항에 따라 달라질 수 있습니다. 빠른 처리가 필요한 경우 크로스인코더 기반 방식이 유용할 수 있으며, 높은 정확도가 중요한 경우에는 고성능 LLM 기반 방식이 더 적합할 수 있습니다. 실제 응용에서는 이 두 방식의 장단점을 고려하여 상황에 맞는 적절한 선택이 필요합니다.